<a href="https://colab.research.google.com/github/tariqulq-cyber/Pemrosesan-teks-praktek/blob/main/Labeling%20data/Labeling_dng_self_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

5220411053_Mouhammad Tariqoul Qalam

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline

In [5]:
df = pd.read_csv('/content/drive/MyDrive/pemrosesan teks teori/labeled_dataa.csv')
df

,userName,at,content,score,thumbsUpCount,content_processed_FINAL,tokenize_review,no_symbol,label
0,YT•RIZ GANT OFFICIAL,10/21/2025 15:36,game nya susah banget anj cuman buat login bik...,1,0,game susah banget anjing cuman login akun suli...,"['game', 'susah', 'banget', 'anjing', 'cuman',...","['game', 'susah', 'banget', 'anjing', 'cuman',...",negative
1,Bang Rama,10/21/2025 14:57,🤑,5,0,NaN,[],[],positive
2,muhammad ridho official,10/21/2025 14:42,not good,1,0,not good,"['not', 'good']","['not', 'good']",negative
3,Fatur Rahman,10/21/2025 13:56,bagus,5,0,bagus,['bagus'],['bagus'],positive
4,rmdn ballboy,10/21/2025 13:23,i really love this game but i'm always disappo...,1,0,really love this game but im always disappoint...,"['really', 'love', 'this', 'game', 'but', 'im'...","['really', 'love', 'this', 'game', 'but', 'im'...",negative
...,...,...,...,...,...,...,...,...,...
945,yohanes okky,9/24/2025 10:35,gem sampah,1,0,game sampah,"['game', 'sampah']","['game', 'sampah']",NaN
946,Pradipta Wicaksono,9/24/2025 10:11,bagus dan seru aku suka,5,0,bagus seru suka,"['bagus', 'seru', 'suka']","['bagus', 'seru', 'suka']",NaN
947,Rahman Latip,9/24/2025 9:54,fix try login again,1,0,fix try login again,"['fix', 'try', 'login', 'again']","['fix', 'try', 'login', 'again']",NaN
948,Fajri Nur Rohman,9/24/2025 7:36,apalah kalo pake wifi ercon mulu kalo pake dat...,5,0,apa wifi ercon mulu data ercon,"['apa', 'wifi', 'ercon', 'mulu', 'data', 'ercon']","['apa', 'wifi', 'ercon', 'mulu', 'data', 'ercon']",NaN


### Langkah 1: Memisahkan Data Berlabel dan Tidak Berlabel & Preprocessing Data
Kita akan memisahkan data frame menjadi dua bagian: satu untuk data yang sudah memiliki label ('positive' atau 'negative') dan satu lagi untuk data yang belum berlabel (labelnya `NaN`). Kita juga akan mengisi nilai `NaN` di kolom `content_processed_FINAL` dengan string kosong agar `TfidfVectorizer` dapat memprosesnya.

In [6]:
import numpy as np

# Handle NaN values in the 'label' column to separate labeled and unlabeled data
labeled_df = df[df['label'].isin(['positive', 'negative'])].copy()
unlabeled_df = df[df['label'].isna()].copy()

# Fill NaN values in 'content_processed_FINAL' with empty strings
labeled_df['content_processed_FINAL'] = labeled_df['content_processed_FINAL'].fillna('')
unlabeled_df['content_processed_FINAL'] = unlabeled_df['content_processed_FINAL'].fillna('')

print(f"Jumlah data berlabel awal: {len(labeled_df)}")
print(f"Jumlah data tidak berlabel awal: {len(unlabeled_df)}")


Jumlah data berlabel awal: 301
Jumlah data tidak berlabel awal: 649


### Langkah 2 & 3: Melatih Model Awal dan Melakukan Self-Training
Sekarang kita akan melatih model awal menggunakan data berlabel. Kemudian, kita akan masuk ke loop self-training di mana model akan memprediksi label untuk data yang belum berlabel. Jika prediksi memiliki kepercayaan tinggi, data tersebut akan ditambahkan ke set pelatihan.

In [25]:
# Inisialisasi daftar untuk menyimpan data pelatihan yang diperluas
X_train_self_training = labeled_df['content_processed_FINAL'].tolist()
y_train_self_training = labeled_df['label'].tolist()

# Salin data tidak berlabel untuk iterasi self-training
unlabeled_texts = unlabeled_df['content_processed_FINAL'].tolist()
unlabeled_indices = unlabeled_df.index.tolist()

# Inisialisasi pipeline model
pipeline = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Parameter self-training
confidence_threshold = 0.75
iterations = 5  # Jumlah iterasi self-training

print("Memulai proses self-training...")

for i in range(iterations):
    print(f"\nIterasi {i+1}:")

    if not unlabeled_texts:
        print("Tidak ada lagi data tidak berlabel yang tersisa. Menghentikan self-training.")
        break

    # Latih model dengan data pelatihan saat ini
    pipeline.fit(X_train_self_training, y_train_self_training)

    # Prediksi probabilitas pada data tidak berlabel
    probabilities = pipeline.predict_proba(unlabeled_texts)
    predictions = pipeline.predict(unlabeled_texts)

    # Temukan prediksi dengan kepercayaan tinggi
    high_confidence_indices = []
    newly_labeled_texts = []
    newly_labeled_labels = []

    for j, (probs, pred_label) in enumerate(zip(probabilities, predictions)):
        max_prob = np.max(probs)
        if max_prob >= confidence_threshold:
            high_confidence_indices.append(j)
            newly_labeled_texts.append(unlabeled_texts[j])
            newly_labeled_labels.append(pred_label)

    if newly_labeled_texts:
        print(f"  Menemukan {len(newly_labeled_texts)} sampel baru dengan kepercayaan tinggi.")
        # Tambahkan sampel baru ke set pelatihan
        X_train_self_training.extend(newly_labeled_texts)
        y_train_self_training.extend(newly_labeled_labels)

        # Hapus sampel yang baru berlabel dari data tidak berlabel
        unlabeled_texts = [text for k, text in enumerate(unlabeled_texts) if k not in high_confidence_indices]
        unlabeled_indices = [idx for k, idx in enumerate(unlabeled_indices) if k not in high_confidence_indices]
        print(f"  Sisa data tidak berlabel: {len(unlabeled_texts)}")
    else:
        print("  Tidak ada sampel baru dengan kepercayaan tinggi yang ditemukan di iterasi ini. Menghentikan self-training.")
        break

print("Self-training selesai.")
print(f"Jumlah data berlabel akhir: {len(X_train_self_training)}")

Memulai proses self-training...

Iterasi 1:
  Menemukan 45 sampel baru dengan kepercayaan tinggi.
  Sisa data tidak berlabel: 604

Iterasi 2:
  Menemukan 77 sampel baru dengan kepercayaan tinggi.
  Sisa data tidak berlabel: 527

Iterasi 3:
  Menemukan 168 sampel baru dengan kepercayaan tinggi.
  Sisa data tidak berlabel: 359

Iterasi 4:
  Menemukan 214 sampel baru dengan kepercayaan tinggi.
  Sisa data tidak berlabel: 145

Iterasi 5:
  Menemukan 98 sampel baru dengan kepercayaan tinggi.
  Sisa data tidak berlabel: 47
Self-training selesai.
Jumlah data berlabel akhir: 903


### Langkah 4: Mengevaluasi Model Akhir
Sekarang kita akan membagi data berlabel awal menjadi set pelatihan dan pengujian untuk mengevaluasi kinerja model akhir yang telah diperkaya melalui self-training. Ini penting untuk mendapatkan gambaran tentang seberapa baik model dapat menggeneralisasi ke data yang tidak terlihat.

In [26]:
# Split the original labeled data into training and testing sets
X_labeled_initial = labeled_df['content_processed_FINAL']
y_labeled_initial = labeled_df['label']

X_train_initial, X_test, y_train_initial, y_test = train_test_split(
    X_labeled_initial,
    y_labeled_initial,
    test_size=0.2,
    random_state=42,
    stratify=y_labeled_initial # Maintain class distribution
)

# Latih ulang model dengan seluruh data yang telah diperluas dari self-training
final_pipeline = make_pipeline(TfidfVectorizer(), MultinomialNB())
final_pipeline.fit(X_train_self_training, y_train_self_training)

# Prediksi pada set pengujian awal (yang belum pernah dilihat oleh model selama self-training)
y_pred = final_pipeline.predict(X_test)

print("\nEvaluasi Model Akhir pada Set Pengujian Awal:")
print(f"Akurasi: {accuracy_score(y_test, y_pred):.4f}")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred))



Evaluasi Model Akhir pada Set Pengujian Awal:
Akurasi: 0.6066

Laporan Klasifikasi:
              precision    recall  f1-score   support

    negative       0.56      1.00      0.71        30
    positive       1.00      0.23      0.37        31

    accuracy                           0.61        61
   macro avg       0.78      0.61      0.54        61
weighted avg       0.78      0.61      0.54        61

